**ChromaDB** is Open Source Vector Database helps users to store their documents in vector form and retrieve relevent documents from Vector DB based on user's query efficiently.
ChromaDB uses Cosine Matrix by default to findout Similarity in query and documents stored in collections.

**Common Distance Metrics for Multimodal Use Cases.**

**01: Cosine Distance (1 - Cosine Similarity)**
      Best For Images, Audio and Texts

**02: L2 Distance (Euclidean)**
      Best for Spatial data, High dimensional embeddings
      Uses whenabsilute magnitude matters

**03: Inner Product (IP)**
      Best for Ranking Based Retrieval
      When using dot-product-based models (e.g., CLIP with max similarity search)

In [ ]:
!pip install chromadb

In [ ]:
!pip install requests


**Chroma DB Client**

**Chroma Clients**

Ephemeral Client

Persistent Client

Client-Server Mode

Python Http-Only Client

In [2]:
import chromadb
chroma_client = chromadb.Client() # Ephemeral Client


**Create Collection**

In [3]:
collection = chroma_client.get_or_create_collection(name="RAG_Application")


**Add External Resources to VectorDB (Chroma DB)**

In [4]:
# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges",
        "My name is M Sheraz Rana and I am 25 years old. I am working on Learning Generative AI development."
    ],
    ids=["id1", "id2","id3"]
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 43.3MiB/s]


**Adding Extra Documents From Extrernal Resources**

In [ ]:
# Assuming each document in the .txt file is separated by a newline
file_path = "/content/PersonalData.txt"  # Path to your .txt file

# Open and read the file
with open(file_path, "r") as file:
    documents = file.readlines()  # Read all lines in the file

# Strip any extra whitespace characters like newlines
documents = [doc.strip() for doc in documents]

# Generate unique ids (or you can assign custom ids)
ids = [f"id{i+1}" for i in range(len(documents))]

# Use the upsert method with the read documents
collection.upsert(
    documents=documents,
    ids=ids
)


**Now, Getting Query and get result from DB**

In [7]:


user_query = "what am I?"
response = collection.query(
    query_texts=user_query, # Chroma will embed this for you
    n_results=2 # how many results to return
)

retrieved_documents = response["documents"][0]
retrieved_documents
context = "\n".join(retrieved_documents)
final_promp = f"Context:\n{context}\n\nUser Query: {user_query}\n Answer:"
print(final_promp)
print(response)

Context:
My name is M Sheraz Rana and I am 25 years old. I am working on Learning Generative AI development.
This is a document about pineapple

User Query: what am I?
 Answer:
{'ids': [['id3', 'id1']], 'embeddings': None, 'documents': [['My name is M Sheraz Rana and I am 25 years old. I am working on Learning Generative AI development.', 'This is a document about pineapple']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.7218676805496216, 1.7450346946716309]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


**Now, Using LLM And observing Response fro it**

In [11]:
import os
import google.generativeai as genai
from google.colab import userdata
key = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=key)

# Create the model
generation_config = {
  "temperature": 1.25,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message(final_promp)

print(response.text)

You are M Sheraz Rana, a 25-year-old who is currently learning generative AI development.

